In [36]:
import pandas as pd
import geopandas as gpd
import numpy as np

from collections import Counter
from functools import reduce
from operator import add
from itertools import chain

import sys

import matplotlib.pyplot as plt

sys.path.append('../utils')
import data_paths
from traffic_counts import TrafficCounts
from hbefa_hot_emissions import HbefaHotEmissions
from excel_calendar import Calendar


In [2]:
# import visum model
visum = gpd.read_file(data_paths.VISUM_FOLDER_PATH + 'visum_links.gpkg')
visum_red = visum.drop(visum.loc[(visum['road_type'] == 'none') | (visum['dtv_SUM']==0)].index, axis = 0)

visum_red.loc[visum_red['road_type']=='Access-residential', 'road_type'] = 'Local/Collector'
visum_red.loc[visum_red['road_type']=='Distributor/Secondary', 'road_type'] = 'TrunkRoad/Primary-National'

In [3]:
# initialize calendar
cal = Calendar()

# initialize traffic cycles
cycles = TrafficCounts()

# initialize HBEFA emission factors
hbefa = HbefaHotEmissions()

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [4]:
# get dates for normweekdays in 2019
dates = cal.get_calendar()
normweekdays_2019 = dates[(dates['day_type'] == 1) &
                          (dates['date'].between('2019-01-01','2019-12-31'))]

In [5]:
# calculate vehicle shares on average norm weekday in 2019
# these values are used to calculate the vehicle share correction factors

weekday_shares_2019 = cycles.vehicle_shares.loc[:,normweekdays_2019['date']].reset_index()
weekday_shares_2019 = weekday_shares_2019.groupby(['road_type','vehicle_class'])[0].mean()
weekday_shares_2019

road_type                   vehicle_class
Access-residential          HGV              0.003515
                            LCV              0.083814
                            MOT              0.044763
                            PC               0.876182
Local/Collector             BUS              0.008581
                            HGV              0.030315
                            LCV              0.099994
                            MOT              0.013732
                            PC               0.847378
Motorway-Nat                BUS              0.004513
                            HGV              0.158198
                            LCV              0.085594
                            MOT              0.003374
                            PC               0.748321
TrunkRoad/Primary-City      BUS              0.002192
                            HGV              0.050612
                            LCV              0.096303
                            MOT         

In [6]:
visum_red['hgv_corr'] = visum_red.apply(lambda row : row['delta_HGV'] / weekday_shares_2019.loc[row['road_type'],'HGV'], axis = 1)
visum_red['lcv_corr'] = visum_red.apply(lambda row : row['delta_LCV'] / weekday_shares_2019.loc[row['road_type'],'LCV'], axis = 1)

In [39]:
date = '2022-05-02'

diurnal_cycles = cycles.get_hourly_scaling_factors(date=date)#.to_dict()
vehicle_shares = cycles.get_vehicle_share(date=date).to_dict()
daily_scaling = cycles.get_daily_scaling_factors(date=date).to_dict()

em_sum_dict = dict()

for row in visum_red.to_dict('records'):

    # get relevant information from the visum model
    road_link_id = row['road_link_id']
    dtv_visum = row['dtv_SUM']
    road_type = row['road_type']
    hour_capacity = row['hour_capacity']
    visum_speed = row['speed']
    visum_slope = row['SLOPE']
    hgv_corr = row['hgv_corr']
    lcv_corr = row['lcv_corr']
    
    # get vehicle shares from counting data
    hgv_share = vehicle_shares['HGV'][road_type]
    lcv_share = vehicle_shares['LCV'][road_type]
    pc_share = vehicle_shares['PC'][road_type]
    mot_share = vehicle_shares['MOT'][road_type]
    bus_share = vehicle_shares['BUS'][road_type]
    
    # calculate vehicle share correction factor
    k = (1- (hgv_corr * hgv_share)- (lcv_corr * lcv_share)) / (1 - hgv_share - lcv_share)
    
    # calculate vehicle counts and apply correction factor
    dtv = dict()
    dtv_day = dtv_visum * daily_scaling[road_type]
    dtv.update({'HGV' : (dtv_day * vehicle_shares['HGV'][road_type] * hgv_corr)})
    dtv.update({'LCV' : (dtv_day * vehicle_shares['LCV'][road_type] * lcv_corr)})
    dtv.update({'PC' : (dtv_day * vehicle_shares['PC'][road_type] * k)})
    dtv.update({'MOT' : (dtv_day * vehicle_shares['MOT'][road_type] * k)})
    dtv.update({'BUS' : (dtv_day * vehicle_shares['BUS'][road_type] * k)})
    
    # prepare array with vehicle counts in the same order as in the diurnal cylces
    # dataframe. This allows to calculate hourly traffic volumes using 
    # vector multiplication
    em = hbefa.calculate_emissions_daily(dtv, diurnal_cycles, road_type,
                                        visum_speed, visum_slope, hour_capacity,  2019)
    
    emission_list = list(chain.from_iterable([list(el.values()) for el in list(em.values())]))
    sum_dict = dict(reduce(add, (map(Counter, emission_list))))
    
    # multiply emission with link lenght
    #TODO
    
    # add daily emission dict (value) and road link id (key) to final emission dict
    em_sum_dict.update({road_link_id:sum_dict})


In [48]:

final_sum = dict(reduce(add, (map(Counter, list(em_sum_dict.values())))))

In [49]:
final_sum

{'CO': 39748241.11610593,
 'NOx': 56034801.73405598,
 'PM': 879980.3046771509,
 'CO2(rep)': 19622172918.890606,
 'CO2(total)': 20624028100.99998,
 'NO2': 10015918.625387514,
 'CH4': 815325.5815994816,
 'BC (exhaust)': 496311.2002348853,
 'CO2e': 19887360994.39458}